# From SCADA to IoT

# About me

# Where do I come from?

In [13]:
!git push origin master

Counting objects: 13, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (11/11), done.
Writing objects: 100% (13/13), 4.18 KiB | 0 bytes/s, done.
Total 13 (delta 0), reused 0 (delta 0)
To git@github.com:D3f0/talks.git
   15f0d63..7436f93  master -> master
